<a href="https://colab.research.google.com/github/odus05/Feature-Engineering/blob/master/feature_tools(nhl_game).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 기법은 항목 당 예측이 필요한 레코드가 많을 때 가장 효과적이라는 것을 알았습니다. 예를 들어 고객 이탈 여부를 예측하는 경우 입력은 각 고객에 대한 세션 이벤트 모음 일 수 있습니다. 사용자 당 하나의 레코드 만있는 경우 심층적 인 기능 합성은 그다지 효과적이지 않습니다. 이 접근 방식의 작동 방식을 보여주기 위해 Kaggle에서 사용할 수있는 NHL 데이터 세트를 사용하겠습니다. 이 데이터 세트에는 게임 테이블과 각 게임을 자세히 설명하는 플레이 테이블이 포함됩니다. 내가 구축하고있는 예측 모델의 목표는 게임 중 플레이 한 내용을 기반으로 플레이 오프 경기를하는 게임을 식별하는 것입니다. 

In [0]:
import pandas as pd
import framequery as fq
import featuretools as ft
from featuretools import Feature
import hashlib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [16]:
data_path = '/content/gdrive/My Drive/Colab Notebooks/datasets/nhl-game-data'
game_df = pd.read_csv(data_path+'/game.csv')
game_df.head()

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
1,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
3,2011030224,20112012,P,2012-05-06,2012-05-06T23:30:00Z,4,1,2,4,home win REG,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
4,2011030225,20112012,P,2012-05-08,2012-05-08T23:30:00Z,1,4,3,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT


In [10]:
plays_df = pd.read_csv(data_path+'/game_plays.csv')

# select a 10% sample of the games
plays_df = fq.execute("""select * from plays_df where game_id % 10 = 1""")

# drop some of the string type fields
plays_df = plays_df.drop(['secondaryType', 'periodType', 'dateTime', 'rink_side'], axis=1).fillna(0)

# convert the remaining strings to integer types via hashing
plays_df['event'] = plays_df['event'].apply(hash)
plays_df['description'] = plays_df['description'].apply(hash)

plays_df.head()

,play_id,game_id,play_num,team_id_for,team_id_against,event,x,y,period,periodTime,periodTimeRemaining,goals_away,goals_home,description,st_x,st_y
0,2011030221_1,2011030221,1,0.0,0.0,-2407464621570790325,0.0,0.0,1,0,1200,0,0,-2407464621570790325,0.0,0.0
1,2011030221_2,2011030221,2,0.0,0.0,-7174183026224657535,0.0,0.0,1,0,1200,0,0,-7174183026224657535,0.0,0.0
2,2011030221_3,2011030221,3,0.0,0.0,310488541993309401,0.0,0.0,1,0,1200,0,0,310488541993309401,0.0,0.0
3,2011030221_4,2011030221,4,4.0,1.0,1560527592483907132,0.0,0.0,1,0,1200,0,0,4700343031429769938,0.0,0.0
4,2011030221_5,2011030221,5,4.0,1.0,4006916355797900616,28.0,24.0,1,21,1179,0,0,3038257804652383570,-28.0,-24.0


In [17]:
# create feature encodings for the event and description fields
es = ft.EntitySet(id='plays')
es = es.entity_from_dataframe(entity_id='plays', 
                              dataframe=plays_df, 
                              index='play_id', 
                              variable_types={'event':ft.variable_types.Categorical, "description": ft.variable_types.Categorical})

f1 = Feature(es['plays']['event'])
f2 = Feature(es['plays']['description'])

encoded, _ = ft.encode_features(plays_df, [f1,f2], top_n=10)
encoded.reset_index(inplace=True)

# create an entry set of the encoded play data and games
es = ft.EntitySet(id='plays')
es = es.entity_from_dataframe(entity_id='plays', dataframe=encoded, index='play_id')
es = es.normalize_entity(base_entity_id='plays', new_entity_id='games', index='game_id')

es

Entityset: plays
  Entities:
    plays [Rows: 383621, Columns: 37]
    games [Rows: 1178, Columns: 1]
  Relationships:
    plays.game_id -> games.game_id

In [20]:
features, defs = ft.dfs(entityset=es, target_entity='games', max_depth=2)
features.reset_index(inplace=True)
features.shape

(1178, 212)

In [26]:
features.head()

,game_id,SUM(plays.index),SUM(plays.event = 1560527592483907132),SUM(plays.event = 3841070161528917008),SUM(plays.event = 3822933506327787810),SUM(plays.event = -9195914453688393760),SUM(plays.event = -6494350409220369269),SUM(plays.event = 812376407238561678),SUM(plays.event = 4006916355797900616),SUM(plays.event = 8733497645179325602),SUM(plays.event = -7728878289795587694),SUM(plays.event = -2309006847487371966),SUM(plays.event is unknown),SUM(plays.description = -7816179181724267305),SUM(plays.description = -9135102062001472723),SUM(plays.description = 2644130248973566963),SUM(plays.description = -33137272110125498),SUM(plays.description = 310488541993309401),SUM(plays.description = -5093118407597457000),SUM(plays.description = -7174183026224657535),SUM(plays.description = 436079302495786491),SUM(plays.description = 8979858467233402383),SUM(plays.description = -2865791206331844756),SUM(plays.description is unknown),SUM(plays.play_num),SUM(plays.team_id_for),SUM(plays.team_id_against),SUM(plays.x),SUM(plays.y),SUM(plays.period),SUM(plays.periodTime),SUM(plays.periodTimeRemaining),SUM(plays.goals_away),SUM(plays.goals_home),SUM(plays.st_x),SUM(plays.st_y),STD(plays.index),STD(plays.event = 1560527592483907132),STD(plays.event = 3841070161528917008),STD(plays.event = 3822933506327787810),...,MIN(plays.goals_home),MIN(plays.st_x),MIN(plays.st_y),MEAN(plays.index),MEAN(plays.event = 1560527592483907132),MEAN(plays.event = 3841070161528917008),MEAN(plays.event = 3822933506327787810),MEAN(plays.event = -9195914453688393760),MEAN(plays.event = -6494350409220369269),MEAN(plays.event = 812376407238561678),MEAN(plays.event = 4006916355797900616),MEAN(plays.event = 8733497645179325602),MEAN(plays.event = -7728878289795587694),MEAN(plays.event = -2309006847487371966),MEAN(plays.event is unknown),MEAN(plays.description = -7816179181724267305),MEAN(plays.description = -9135102062001472723),MEAN(plays.description = 2644130248973566963),MEAN(plays.description = -33137272110125498),MEAN(plays.description = 310488541993309401),MEAN(plays.description = -5093118407597457000),MEAN(plays.description = -7174183026224657535),MEAN(plays.description = 436079302495786491),MEAN(plays.description = 8979858467233402383),MEAN(plays.description = -2865791206331844756),MEAN(plays.description is unknown),MEAN(plays.play_num),MEAN(plays.team_id_for),MEAN(plays.team_id_against),MEAN(plays.x),MEAN(plays.y),MEAN(plays.period),MEAN(plays.periodTime),MEAN(plays.periodTimeRemaining),MEAN(plays.goals_away),MEAN(plays.goals_home),MEAN(plays.st_x),MEAN(plays.st_y),COUNT(plays),label
0,2010020001,1097336604,43,47,61,31,43,24,23,13,8,5,14,12,3,2,4,3,3,3,0,2,3,277,48828,2390.0,2416.0,-1877.0,58.0,605,184640,189760,318,622,77.0,-434.0,90.210864,0.345265,0.358273,0.397232,...,0,-98.0,-41.0,3517104.5,0.137821,0.150641,0.195513,0.099359,0.137821,0.076923,0.073718,0.041667,0.025641,0.016026,0.044872,0.038462,0.009615,0.006410,0.012821,0.009615,0.009615,0.009615,0.000000,0.006410,0.009615,0.887821,156.5,7.660256,7.743590,-6.016026,0.185897,1.939103,591.794872,608.205128,1.019231,1.993590,0.246795,-1.391026,312,0
1,2010020011,1005644016,56,56,45,46,35,16,18,30,11,6,14,18,5,5,7,3,3,3,0,1,3,285,55611,3519.0,3579.0,1979.0,-875.0,655,200288,199312,373,555,1391.0,41.0,96.273049,0.374578,0.374578,0.342382,...,0,-96.0,-41.0,3019952.0,0.168168,0.168168,0.135135,0.138138,0.105105,0.048048,0.054054,0.090090,0.033033,0.018018,0.042042,0.054054,0.015015,0.015015,0.021021,0.009009,0.009009,0.009009,0.000000,0.003003,0.009009,0.855856,167.0,10.567568,10.747748,5.942943,-2.627628,1.966967,601.465465,598.534535,1.120120,1.666667,4.177177,0.123123,333,0
2,2010020021,925290853,58,49,59,45,30,20,21,16,9,5,14,15,9,9,6,3,3,3,0,0,0,278,53301,4391.0,4420.0,-536.0,550.0,645,189750,201450,532,427,1454.0,-512.0,94.252321,0.383028,0.357921,0.385595,...,0,-99.0,-39.0,2838315.5,0.177914,0.150307,0.180982,0.138037,0.092025,0.061350,0.064417,0.049080,0.027607,0.015337,0.042945,0.046012,0.027607,0.027607,0.0

In [22]:
# assign labels to the generated features
features = fq.execute("""select f.*, 
                         case when g.type='P' then 1 else 0 end as label
                         from features f
                         join game_df g
                         on f.game_id = g.game_id""")
# count the labels
fq.execute("""select label, sum(1) as plays 
              from features
              group by label""")

,label,plays
0,0,1044
1,1,134


In [25]:
y = features['label']
X = features.drop(['label', 'game_id'], axis=1).fillna(0)

lr = LogisticRegression()
model = lr.fit(X, y)
print('accuracy_score: ', model.score(X, y))
print('roc_auc_score: ', roc_auc_score(y, model.predict_proba(X)[:, 1]))


accuracy_score:  0.9592529711375212
roc_auc_score:  0.9190327100131527


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
